In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Image size and batch size
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Set up ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Training data generator
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # for multi-class classification
    subset='training'
)

# Validation data generator
val_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)



Found 32 images belonging to 4 classes.
Found 8 images belonging to 4 classes.


In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes for each tumor type
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

In [14]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Start with a moderate number of epochs
    steps_per_epoch=train_data.samples // BATCH_SIZE,
    validation_steps=val_data.samples // BATCH_SIZE
)


Epoch 1/20
1/1 [==============================] - 1s 870ms/step - loss: 1.3905 - accuracy: 0.1562
Epoch 2/20
1/1 [==============================] - 0s 308ms/step - loss: 1.4700 - accuracy: 0.3438
Epoch 3/20
1/1 [==============================] - 0s 377ms/step - loss: 1.4680 - accuracy: 0.2812
Epoch 4/20
1/1 [==============================] - 0s 342ms/step - loss: 1.4074 - accuracy: 0.1562
Epoch 5/20
1/1 [==============================] - 0s 368ms/step - loss: 1.4067 - accuracy: 0.1562
Epoch 6/20
1/1 [==============================] - 0s 364ms/step - loss: 1.3739 - accuracy: 0.3125
Epoch 7/20
1/1 [==============================] - 0s 462ms/step - loss: 1.3911 - accuracy: 0.2188
Epoch 8/20
1/1 [==============================] - 0s 345ms/step - loss: 1.3739 - accuracy: 0.2188
Epoch 9/20
1/1 [==============================] - 0s 396ms/step - loss: 1.3574 - accuracy: 0.3750
Epoch 10/20
1/1 [==============================] - 0s 356ms/step - loss: 1.3601 - accuracy: 0.3750
Epoch 11/20
1/1 [==

In [15]:
# Save the trained model
model.save('brain_tumor_classifier.h5')


In [16]:
# Load the testing data
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy}")


Found 40 images belonging to 4 classes.
2/2 [==============================] - 0s 14ms/step - loss: 1.7989 - accuracy: 0.2250
Test Accuracy: 0.22499999403953552


In [17]:
# Paths to training and testing directories
train_dir = r"C:\Users\tadak\OneDrive\Desktop\Data analysis\work\Challenge\Project 4\Project4\Eric's Work\Sample_BrainTumor_images\Sample_BrainTumor_images\Training"
test_dir = r"C:\Users\tadak\OneDrive\Desktop\Data analysis\work\Challenge\Project 4\Project4\Eric's Work\Sample_BrainTumor_images\Sample_BrainTumor_images\Testing"

# Updated categories with underscores
categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']



In [20]:
# Define a function to preprocess images and make predictions
def process_and_predict_images(directory, model, categories):
    results = []  # List to store prediction results
    
    IMG_SIZE = (128, 128)  # Size expected by the model
    
    # Loop through each category
    for category in categories:
        category_path = os.path.join(directory, category)
        
 # Check if the category folder exists
 if not os.path.exists(category_path):
print(f"Folder not found: {category_path}")
continue  # Skip to the next category if folder is missing
        
 # Loop through each image in the category folder
 for img_name in os.listdir(category_path):
     img_path = os.path.join(category_path, img_name)
            
     # Load and preprocess the image
     img = cv2.imread(img_path)
    if img is None:
  print(f"Could not load image: {img_path}")
         continue
            
         img = cv2.resize(img, IMG_SIZE)
         img = img_to_array(img) / 255.0
         img = np.expand_dims(img, axis=0)  # Add batch dimension
            
  # Make prediction
    prediction = model.predict(img)
     predicted_class_index = np.argmax(prediction)  # Index of highest probability
     predicted_label = categories[predicted_class_index]  # Map index to category name
  
     # Append results
     results.append({
        "Filename": img_name,
           "True Label": category,
           "Predicted Label": predicted_label,
           "Probability": float(prediction[0][predicted_class_index])  # Highest probability value
          })
    
return results


In [21]:
train_results = process_and_predict_images(train_dir, model, categories)
test_results = process_and_predict_images(test_dir, model, categories)

1/1 [==============================] - 0s 18ms/step


In [24]:
# Convert results to DataFrames
train_results_df = pd.DataFrame(train_results)
test_results_df = pd.DataFrame(test_results)

# Display the DataFrames as tables
print("Training Results:")
display(train_results_df)  # This will display the table in Jupyter Notebook

print("\nTesting Results:")
display(test_results_df)  # This will display the table in Jupyter Notebook

# Save results to CSV files
train_results_df.to_csv('train_predictions_log.csv', index=False)
test_results_df.to_csv('test_predictions_log.csv', index=False)

print("Training and testing results saved to CSV files.")

Training Results:


,Filename,True Label,Predicted Label,Probability
0,gg (156).jpg,glioma_tumor,glioma_tumor,0.342086
1,gg (170).jpg,glioma_tumor,pituitary_tumor,0.309551
2,gg (328).jpg,glioma_tumor,glioma_tumor,0.333892
3,gg (360).jpg,glioma_tumor,pituitary_tumor,0.313400
4,gg (468).jpg,glioma_tumor,glioma_tumor,0.412532
5,gg (609).jpg,glioma_tumor,glioma_tumor,0.470238
6,gg (65).jpg,glioma_tumor,pituitary_tumor,0.336608
7,gg (703).jpg,glioma_tumor,glioma_tumor,0.453854
8,gg (747).jpg,glioma_tumor,glioma_tumor,0.447953
9,gg (82).jpg,glioma_tumor,no_tumor,0.325137



Testing Results:


,Filename,True Label,Predicted Label,Probability
0,image(19).jpg,glioma_tumor,glioma_tumor,0.315917
1,image(21).jpg,glioma_tumor,no_tumor,0.532678
2,image(26).jpg,glioma_tumor,glioma_tumor,0.395945
3,image(31).jpg,glioma_tumor,pituitary_tumor,0.349719
4,image(36).jpg,glioma_tumor,pituitary_tumor,0.338290
5,image(61).jpg,glioma_tumor,glioma_tumor,0.398685
6,image(74).jpg,glioma_tumor,glioma_tumor,0.412503
7,image(78).jpg,glioma_tumor,no_tumor,0.528907
8,image(87).jpg,glioma_tumor,no_tumor,0.562415
9,image(94).jpg,glioma_tumor,no_tumor,0.487033


Training and testing results saved to CSV files.
